In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from main import parse_config, instantiate_agents, instantiate_auction

In [ ]:
# Parse configuration file
rng, config, agent_configs, agents2items, agents2item_values,\
num_runs, max_slots, embedding_size, embedding_var,\
obs_embedding_size = parse_config('../auction-gym/config/SP_Oracle.json')

# Config

For an explanation of the config fields, see [CONFIG.md](https://github.com/amzn/auction-gym/blob/065f8bf325ebbec9c96631625ef1c36df3870cb3/CONFIG.md?plain=1#L30).

In [ ]:
config

In [ ]:
agent_configs

In [ ]:
agents = instantiate_agents(rng, agent_configs, agents2item_values, agents2items)

auction, num_iter, rounds_per_iter, output_dir =\
    instantiate_auction(rng,
                        config,
                        agents2items,
                        agents2item_values,
                        agents,
                        max_slots,
                        embedding_size,
                        embedding_var,
                        obs_embedding_size)
config

In [ ]:
# Let's decrease rounds_per_iter for now b/c we just want to test things out.
# Using a lower rounds_per_iter just means that the runs will take less time.
rounds_per_iter = 100

In [ ]:
for _ in range(rounds_per_iter):
    auction.simulate_opportunity()

In [ ]:
agents[0].net_utility